In [4]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import numpy as np

import pickle
from scipy.integrate import simps
from scipy.interpolate import RectBivariateSpline
from classy import Class
import sys
sys.path.append("../scripts/") 
from postBornCrossSpectra import PostBorn_Bispec
import Cosmology as C
import kernels
plt.style.use('classic')
plt.style.use(['seaborn-colorblind','paper'])


In [5]:

from lab import *

In [6]:
plotpath = '../output/plots/'

In [7]:
zmin = 1e-5
zmax = 1090.

kmin = 1e-4
kmax = 50

cosmo_dict.pop('z_cmb')
cosmo = [{'name':"EmasCosmo"},cosmo_dict]

print(cosmo_dict)

a     = np.linspace(1./(1.+zmin),1./(1.+zmax),500)  
z     = 1./a-1.

data  = C.CosmoData(cosmo[1],z)

L     = np.logspace(1,4,400)

# sampling for integrations
phi = np.linspace(0.,2.*np.pi,1000,endpoint=True)
L1  = np.logspace(-1,5,400)

LSST_bins  = [0,1,2,3,4,'all']

{u'A_s': 2.10732e-09, u'h': 0.677, u'n_s': 0.96824, u'k_pivot': 0.05, u'omega_b': 0.02247, u'omega_cdm': 0.11923}
computing distances, derived parameters...
{'Neff': 3.046}
{'h': 0.677}
{'m_ncdm_tot': 0.0}
{'omega_m': 0.6745515980724002}


In [8]:
# define redshift bins (LSST-like)
zmins= [0,0.5,1,2,3,0]
zmaxs= [0.5,1.,2.,3.,4.,'z$_{\mathrm{CMB}}$']
ngal = [14.3,23.9,21.6,3.7,1.5,65.7]

colors=['navy','maroon', 'darkgreen', 'mediumaquamarine','indianred','darkorange']

In [9]:
#compute correction following Krause&Hirata
#always use trapz rule for accurate results!
#redefinition with l'-> l'-l
def get_regularized_sum_of_terms(L,L1,phi,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            L_= LL*np.sqrt(1.+(LL1/LL)**2+2.*LL1/LL*nu)
            integrand =LL*LL1**3*(LL1*nu+LL)*(LL1+LL*nu)**2*(M(L_,LL1,grid=False)/L_**2/LL1**4-M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res


#numerical angular integration
def get_second_sum_of_terms(L,L1,M,method='trapz'):
    nu = np.cos(phi)
    res=[]
    for LL in L:
        resL1=[]
        for LL1 in L1:
            integrand = LL1*(LL*LL1**2*(-LL1*nu+LL)*(-LL1+LL*nu)**2-LL**2*(LL*LL1*nu)**2)*(M(LL,LL1,grid=False)/LL**2/LL1**4)
            if method =='simps':
                resL1+=[simps(integrand,phi)]
            elif method =='trapz':
                resL1+=[np.trapz(integrand,phi)]
            elif method =='sum':
                resL1+=[np.sum(integrand*np.diff(phi)[0])]
        res+=[simps(resL1,L1)]
    res=4.*np.asarray(res)/(2*np.pi)**2
    return res

In [10]:
first_kernel  = kernels.CMB_lens(data.chi_cmb,data)
simple_kernel = kernels.CMB_lens(None,data)

Mstarspls = []
PBs =[]
Cls= []
for LSST_bin in LSST_bins:
    second_kernel = kernels.gal_clus(kernels.dNdz_LSST,kernels.simple_bias,data,LSST_bin)
    PB  = PostBorn_Bispec(data, zmin, data.z_cmb, first_kernel, second_kernel, simple_kernel, k_min=kmin,k_max=100, lmax=30000, acc=2)
    PBs+=[PB]
    Mstarspls+=[PB.Mstarsp]
    Cls+=[PB.CL_born]
ls = PB.ls


using z-bin [0, 0.5, 0.03] norm 14.3227605664
Initializing CLASS with halofit...
{'k_step_super': 0.0001, 'perturb_sampling_stepsize': 0.01, u'A_s': 2.10732e-09, 'k_max_tau0_over_l_max': 5.0, 'k_min_tau0': 0.002, u'h': 0.677, 'z_max_pk': 1088.6464513003543, 'k_step_super_reduction': 0.1, 'non linear': 'halofit', u'omega_cdm': 0.11923, 'tol_perturb_integration': 1e-06, u'k_pivot': 0.05, 'k_per_decade_for_pk': 20, 'P_k_max_1/Mpc': 101, 'output': 'mPk', u'omega_b': 0.02247, u'n_s': 0.96824, 'k_step_sub': 0.015, 'halofit_k_per_decade': 3000.0}
sigma8  0.822427246139
('zmax', 1088.6464513003543)
using z-bin [0.5, 0.5, 0.03] norm 23.9432092389
Initializing CLASS with halofit...
{'k_step_super': 0.0001, 'perturb_sampling_stepsize': 0.01, u'A_s': 2.10732e-09, 'k_max_tau0_over_l_max': 5.0, 'k_min_tau0': 0.002, u'h': 0.677, 'z_max_pk': 1088.6464513003543, 'k_step_super_reduction': 0.1, 'non linear': 'halofit', u'omega_cdm': 0.11923, 'tol_perturb_integration': 1e-06, u'k_pivot': 0.05, 'k_per_deca

In [13]:
corrs=[]
for ii in range(len(LSST_bins)):
    print(ii),
    corrs+=[get_regularized_sum_of_terms(L,L1,phi,Mstarspls[ii],method='trapz')+get_second_sum_of_terms(L,L1,Mstarspls[ii],method='trapz')]

 0 1 2 3 4 5


In [14]:
corrs = np.asarray(corrs)

In [15]:
corrs

(6, 400)

In [17]:
np.save('../output/LimberPostBornResultsLSST',(ls,corrs))


In [19]:
ls,corrs = np.load('../output/LimberPostBornResultsLSST.npy')

In [20]:
ls

array([  1.00000000e-03,   1.14815362e-03,   1.31825674e-03, ...,
         2.99500000e+04,   2.99750000e+04,   3.00000000e+04])

In [22]:
corrs.shape

(6, 400)